## Import Libraries

In [97]:
import cv2
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
import re
from PIL import ImageEnhance , Image
from google.colab.patches import cv2_imshow

## SimpleBlobDetection Parameters

In [98]:
# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()
 
# Change thresholds
params.minThreshold = 150;
params.maxThreshold = 255;
 
# Filter by Area.
params.filterByArea = True
params.minArea = 50
 
# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.3
 
# Filter by Convexity
params.filterByConvexity = True
params.minConvexity = 0.7
 
# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.09
 
# Create a detector with the parameters
ver = (cv2.__version__).split('.')
if int(ver[0]) < 3 :
  detector = cv2.SimpleBlobDetector(params)
else : 
  detector = cv2.SimpleBlobDetector_create(params)

## SimpleBlobDetection Algorithm

In [99]:
def Blob_image(img,detector):

  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  blur = cv2.GaussianBlur(img_rgb, (11,11), 0)  # improve

  lab = cv2.cvtColor(blur, cv2.COLOR_BGR2LAB)

  l_component = lab[:, :, 0]

  keypoints= detector.detect(l_component)

  #blob_sizes = [kp.size for kp in keypoints]
  img = img.copy()
  for x in range(1,len(keypoints)):
    img=cv2.circle(img, (np.int(keypoints[x].pt[0]),np.int(keypoints[x].pt[1])), radius=np.int(keypoints[x].size), color=(0,0,0), thickness=-1)

  return img 

## AdaptiveThresholding Algorithm

In [100]:
def map_img(blob_img,original_image):
  
  kernel = np.ones((7,7), np.uint8)

  # Apply dilation
  dilated_img = cv2.dilate(blob_img, kernel, iterations=1)

  gray = cv2.cvtColor(dilated_img, cv2.COLOR_BGR2GRAY) 

  # Apply Gaussian blurring
  blurred = cv2.GaussianBlur(gray, (7, 7), 0)
  
  #thresholding
  thresholdImage = cv2.adaptiveThreshold(blurred, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY ,11,3)

  # Detect edges using Canny Edge Detection
  edges = cv2.Canny(thresholdImage, 140, 255,5)

  # Find contours in the image
  contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Filter the contours
  filtered_contours = []
  for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    if h < 120 and w < 120:
      #filtered_contours.append(contour)
      if cv2.contourArea(contour) > 100:
        filtered_contours.append(contour)

  # Draw rectangles around the contours
  for contour in filtered_contours:
      x, y, w, h = cv2.boundingRect(contour)
      cv2.rectangle(blob_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
      cv2.rectangle(original_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

  return blob_img, original_image, edges, thresholdImage

## Running the defined algorithms 

In [ ]:
filepath = '/content/drive/MyDrive/Project_Images/Remapped_Images/'
for i in range(1,13):
  img_101 = cv2.imread('/content/drive/MyDrive/Project_Images/Photos/mole_img_{0}.jpg'.format(i))
  blob_image = Blob_image(img_101,detector)
  blob_img, original_image, edges, thresholdImage = map_img(blob_image,img_101)

  cv2.imwrite(os.path.join(filepath, 'mapped_image_{0}.jpg'.format(i)), original_image)